In [13]:
!python train_noddp.py 

14 loss functions to train with
3 models to train with

Train raw: 712 Train ref: 712 
Test raw: 178 Test ref: 178

Training: UNet with PerceptualLoss_vgg11
100%|█████████████████████████████████████████| 100/100 [23:31<00:00, 14.12s/it]
Testando o modelo UNet_PerceptualLoss_vgg11
0it [00:00, ?it/s]
Traceback (most recent call last):
  File "/home/pdi_4/Documentos/Loss_battle/train_noddp.py", line 91, in <module>
    train_models(epochs=args.epochs)
  File "/home/pdi_4/Documentos/Loss_battle/train_noddp.py", line 69, in train_models
    psnr_value, ssim_value, uciqe_, uiqm = calculate_metrics(predictions, target)
                                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/pdi_4/Documentos/Loss_battle/metrics/metrics.py", line 472, in calculate_metrics
    uciqe_ = uciqe(nargin=1,loc=corrected)#usarei este
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/pdi_4/Documentos/Loss_battle/metrics/metrics.py", line 39, in uciqe
    img_lab = cv2.cvtC

In [15]:
# Definir a string do caminho
path = "output/ckpt_battle/UNet_PerceptualLoss_vgg11_ckpt.pth"

# Dividir a string usando '/' como delimitador e selecionar o último elemento
filename = path.split('/')[-1].split('.')[0]

# Imprimir o resultado
print(filename)  

UNet_PerceptualLoss_vgg11_ckpt


### torch run multi gpu

In [ ]:
!torchrun --nnodes=2 --nproc_per_node=2 --node_rank=0 --master_addr="10.228.252.209" --master_port=22 main.py --nodes=2 --gpus=2 --epochs=100

### torchrun local

In [ ]:
!torchrun --nproc_per_node=2 main.py --nodes=1 --gpus=2 --epochs=100